# Option Pricing and Stochastic Volatility using Neural Network

## Testing Data Collection

In [5]:
from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override()

european_index = pdr.get_data_yahoo('VESIX', '2020-01-01', '2021-01-01')
print(european_index)

[*********************100%***********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close  Volume
Date                                                                     
2020-01-02  31.400000  31.400000  31.400000  31.400000  28.110714       0
2020-01-03  31.160000  31.160000  31.160000  31.160000  27.895857       0
2020-01-06  31.209999  31.209999  31.209999  31.209999  27.940619       0
2020-01-07  31.059999  31.059999  31.059999  31.059999  27.806332       0
2020-01-08  31.080000  31.080000  31.080000  31.080000  27.824236       0
...               ...        ...        ...        ...        ...     ...
2020-12-24  32.049999  32.049999  32.049999  32.049999  29.422323       0
2020-12-28  32.310001  32.310001  32.310001  32.310001  29.661007       0
2020-12-29  32.560001  32.560001  32.560001  32.560001  29.890509       0
2020-12-30  32.549999  32.549999  32.549999  32.549999  29.881327       0
2020-12-31  32.360001  32.360001  32.360001

## Training Data Collection

In [10]:
import pandas as pd

nIn = 31
nOut = 500
nXi = 28

data_train = pd.read_csv('data/heston_train.csv', delimiter=",").values
x_train = data_train[:,:nIn]
y_train = data_train[:,nIn:nIn+nOut]
data_train = None

data_test = pd.read_csv('data/heston_test.csv', delimiter=",").values
x_valid = data_test[:,:nIn]
y_valid = data_test[:,nIn:nIn+nOut]
data_test = None


## Data Pre-Processing

In [11]:
from sklearn.preprocessing import StandardScaler
import numpy as np

tmp1 = np.reshape(np.array([0.50,1.25,0.00]), (1, 3))
tmp2 = np.reshape(np.array([0.00,0.10,-1.00]), (1, 3))
ub = np.concatenate((tmp1,np.tile(1,(1,nXi))),1)
lb = np.concatenate((tmp2,np.tile(0.0025,(1,nXi))),1)

def scale(x: list):
    res=np.zeros(nIn)
    for i in range(nIn):
        res[i]=(x[i] - (ub[0,i] + lb[0,i])*0.5) * 2 / (ub[0,i] - lb[0,i])
    return res

x_train = np.array([scale(x) for x in x_train])
x_test = np.array([scale(x) for x in x_valid])

scale_y =  StandardScaler()
y_train = scale_y.fit_transform(y_train)
y_test = scale_y.transform(y_valid)

## Neural Network Implementation

In [12]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(
    hidden_layer_sizes=(100,100,100,100),
    solver='adam',
    shuffle = False,
    batch_size=64,
    verbose=True,
    max_iter= 25
)

## Network Training

In [13]:
import random
random.seed(1926159)

model.fit(x_train, y_train)

/Users/harvey/Documents/Portfolio/Option Pricing and Stochastic Volatility/environment/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPRegressor(batch_size=64, hidden_layer_sizes=(100, 100, 100, 100),
             max_iter=25, shuffle=False, verbose=True)

In [ ]:
import pickle

filename = 'neural_network_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
filename = 'neural_network_model.sav'
model = pickle.load(open(filename, 'rb'))

## Testing And Analysis

In [1]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from scipy import stats

y_predicted = model.predict(x_test)

def performance_analysis(true: list, predicted: list):
    mse = mean_squared_error(true, predicted)
    rmse = sqrt(mse)
    mae = mean_absolute_error(true, predicted)
    r2 = r2_score(true, predicted)
    return mse, rmse, mae, r2


SyntaxError: incomplete input (4135998226.py, line 7)

In [ ]:
from heston_monte_carlo import heston_model_monte_carlo_simulation


